In [1]:
import cv2
import numpy as np
import random
import time
from deepface import DeepFace
from mtcnn.mtcnn import MTCNN

# Configuración de los modelos de detección de caras
haar_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
dnn_model = "deploy.prototxt.txt"
dnn_weights = "res10_300x300_ssd_iter_140000.caffemodel"
net = cv2.dnn.readNetFromCaffe(dnn_model, dnn_weights)
mtcnn_det = MTCNN()

# Cargar emojis para cada emoción
emojisDict = {
    "happy": cv2.imread("images/happy.png", cv2.IMREAD_UNCHANGED),
    "sad": cv2.imread("images/sad.png", cv2.IMREAD_UNCHANGED),
    "fear": cv2.imread("images/fear.png", cv2.IMREAD_UNCHANGED),
    "neutral": cv2.imread("images/neutral.png", cv2.IMREAD_UNCHANGED),
    "disgust": cv2.imread("images/disgust.png", cv2.IMREAD_UNCHANGED),
    "angry": cv2.imread("images/angry.png", cv2.IMREAD_UNCHANGED),
    "surprise": cv2.imread("images/surprised.png", cv2.IMREAD_UNCHANGED)
}

# Definir emociones e instrucciones
emotionsList = list(emojisDict.keys())
score = 0
countdown_time = 8
time_to_react = 2 # you have two seconds to change your emotion before it is counted

# Conexión a la cámara
video_capture = cv2.VideoCapture(0)

# Variables del juego
current_emotion = random.choice(emotionsList)
instruction_text = f"Simon says: Be {current_emotion}"
start_time = time.time()
det = 0  # Índice para cambiar entre detectores

# Bucle principal del juego
while True:
    ret, frame = video_capture.read()
    if not ret:
        break

    # Calcular el tiempo transcurrido para la cuenta regresiva
    elapsed_time = time.time() - start_time
    remaining_time = countdown_time - int(elapsed_time)

    # Reiniciar el juego después de cada cuenta regresiva
    if remaining_time <= 0:
        remaining_time = countdown_time
        start_time = time.time()
        
        # Decidir si la siguiente instrucción es una trampa
        is_trap = random.randint(0, 1)
        current_emotion = random.choice(emotionsList)
        instruction_text = f"Be {current_emotion}" if is_trap else f"Simon says: Be {current_emotion}"

    # Cambiar de detector de caras según el valor de `det`
    faces = []
    if det == 0:
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        faces = haar_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))
    elif det == 1:
        h, w = frame.shape[:2]
        blob = cv2.dnn.blobFromImage(frame, 1.0, (300, 300), [104.0, 177.0, 123.0], False, False)
        net.setInput(blob)
        detections = net.forward()
        for i in range(detections.shape[2]):
            confidence = detections[0, 0, i, 2]
            if confidence > 0.5:
                box = detections[0, 0, i, 3:7] * np.array([w, h, w, h])
                (x, y, x2, y2) = box.astype("int")
                faces.append((x, y, x2 - x, y2 - y))
    else: # case det == 2
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = mtcnn_det.detect_faces(rgb_frame)
        faces = [(result['box'][0], result['box'][1], result['box'][2], result['box'][3]) for result in results]

    if remaining_time <= (countdown_time - time_to_react):
        # Detectar emociones y superponer el emoji correspondiente para cada cara
        for (x, y, w, h) in faces:
            face_img = frame[y:y+h, x:x+w]
            try:
                # Analizar la emoción de la cara usando DeepFace
                analysis = DeepFace.analyze(img_path=face_img, enforce_detection=False, actions=['emotion'])
                detected_emotion = analysis[0]['dominant_emotion']
                
                # Superponer el emoji de la emoción detectada
                if detected_emotion in emotionsList:
                    emoji = emojisDict[detected_emotion]
                    emoji_resized = cv2.resize(emoji, (w, h))

                    # Superponer el emoji con transparencia
                    for i in range(h):
                        for j in range(w):
                            if emoji_resized[i, j][3] != 0:
                                frame[y + i, x + j] = emoji_resized[i, j][:3]
                
                # Comprobar si la emoción detectada coincide con la instrucción actual
                if detected_emotion == current_emotion:
                    if not is_trap:
                    #if "Simon says" in instruction_text:
                        score += 1  # Respuesta correcta
                    else:
                        score -= 1  # Respuesta incorrecta por trampa
                    remaining_time = countdown_time  # Reiniciar cuenta regresiva
                    start_time = time.time()
                    current_emotion = random.choice(emotionsList)
                    is_trap = random.randint(0, 1)
                    instruction_text = f"Be {current_emotion}" if is_trap else f"Simon says: Be {current_emotion}"

            except Exception as e:
                print(f"Fallo en la detección de emoción para la cara en {(x, y, w, h)}: {e}")

    # Mostrar la instrucción actual
    cv2.rectangle(frame, (0, frame.shape[0] - 50), (frame.shape[1], frame.shape[0]), (255, 255, 255), -1)
    cv2.putText(frame, instruction_text, (10, frame.shape[0] - 15), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 0), 2)

    # Dibujar cuenta regresiva en la esquina superior derecha
    cv2.circle(frame, (frame.shape[1] - 60, 60), 30, (169, 169, 169), -1)
    cv2.putText(frame, str(remaining_time), (frame.shape[1] - 75, 75), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Mostrar puntaje en la esquina superior izquierda
    cv2.putText(frame, f"Score: {score}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2)

    # Mostrar el resultado en pantalla
    cv2.imshow('Simon Says Emotion Game', frame)

    # Cambiar de detector con 'd', salir con 'Esc'
    key = cv2.waitKey(1) & 0xFF
    if key == ord('d'):
        det = (det + 1) % 3
    elif key == 27:
        break

# Liberar la cámara y cerrar ventanas
video_capture.release()
cv2.destroyAllWindows()
